# Production Deployment & Monitoring - School Suspension Prediction

**Purpose:** Prepare the model for production deployment with FastAPI and monitoring

**Critical:** This notebook creates production artifacts and monitoring tools

**Structure:**
- Section 0: Setup & load production model
- Section 1: Create prediction pipeline
- Section 2: Input validation & preprocessing
- Section 3: Batch prediction capabilities
- Section 4: Model versioning & metadata
- Section 5: Performance monitoring tools
- Section 6: Generate FastAPI endpoints
- Section 7: Docker & deployment artifacts

**References:**
- 02_eda_and_core_model_all_features.ipynb (training)
- 03_final_test_evaluation.ipynb (validation)
- ../web/app.py (FastAPI integration)

## Section 0: Setup & Configuration

In [ ]:
# Cell 0.1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
from pathlib import Path
import json
import joblib
import pickle
from typing import Dict, List, Any, Optional

# Validation
from pydantic import BaseModel, Field, validator

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Paths
DATA_DIR = Path('../data')
PROCESSED_DIR = DATA_DIR / 'processed'
MODELS_DIR = Path('../models')
MODELS_DIR.mkdir(exist_ok=True)
PRODUCTION_DIR = MODELS_DIR / 'production'
PRODUCTION_DIR.mkdir(exist_ok=True)

# LOCATION_MAPPING
LOCATION_MAPPING = {
    0: 'Manila', 1: 'Quezon City', 2: 'Caloocan', 3: 'Las Piñas',
    4: 'Makati', 5: 'Malabon', 6: 'Mandaluyong', 7: 'Marikina',
    8: 'Muntinlupa', 9: 'Navotas', 10: 'Parañaque', 11: 'Pasay',
    12: 'Pasig', 13: 'Pateros', 14: 'San Juan', 15: 'Taguig',
    16: 'Valenzuela'
}

REVERSE_LOCATION_MAPPING = {v: k for k, v in LOCATION_MAPPING.items()}

print("✅ Libraries imported successfully")
print(f"Production directory: {PRODUCTION_DIR}")

In [ ]:
# Cell 0.2: Load Best Model and Metadata
print("="*80)
print("LOADING PRODUCTION MODEL")
print("="*80)

# Load metadata
with open(str(PROCESSED_DIR / 'core_model_metadata.json'), 'r') as f:
    metadata = json.load(f)

print(f"\nModel: {metadata['best_model']}")
print(f"  F2 Score: {metadata['best_f2']:.4f}")
print(f"  Recall: {metadata['best_recall']:.4f}")
print(f"  Precision: {metadata['best_precision']:.4f}")
print(f"  Features: {metadata['final_feature_count']}")

# Load model
model_path = PROCESSED_DIR / 'best_core_model.pkl'
best_model = joblib.load(str(model_path))
print(f"\n✅ Model loaded: {type(best_model).__name__}")

# Store expected features
expected_features = metadata['selected_features']
print(f"\nExpected features: {len(expected_features)}")

## Section 1: Create Prediction Pipeline

In [ ]:
# Cell 1.1: Define Pydantic Models for Input Validation
from pydantic import BaseModel, Field, validator
from typing import Optional
from datetime import date

class PredictionInput(BaseModel):
    """Input schema for single prediction request"""
    
    # Date and location
    date: str = Field(..., description="Date in YYYY-MM-DD format")
    lgu_name: str = Field(..., description="LGU name (e.g., 'Manila', 'Quezon City')")
    
    # Calendar features (can be auto-computed from date)
    is_holiday: Optional[int] = Field(None, description="1 if holiday, 0 otherwise")
    is_school_day: Optional[int] = Field(None, description="1 if school day, 0 otherwise")
    
    # Weather features (historical - day before)
    hist_precipitation_sum_t1: float = Field(..., description="Precipitation sum (mm) - previous day")
    hist_wind_speed_max_t1: float = Field(..., description="Max wind speed (km/h) - previous day")
    hist_wind_gusts_max_t1: Optional[float] = Field(None, description="Max wind gusts (km/h) - previous day")
    hist_pressure_msl_min_t1: Optional[float] = Field(None, description="Min MSL pressure (hPa) - previous day")
    hist_temperature_max_t1: Optional[float] = Field(None, description="Max temperature (°C) - previous day")
    hist_relative_humidity_mean_t1: Optional[float] = Field(None, description="Mean relative humidity (%) - previous day")
    hist_cloud_cover_max_t1: Optional[float] = Field(None, description="Max cloud cover (%) - previous day")
    hist_dew_point_mean_t1: Optional[float] = Field(None, description="Mean dew point (°C) - previous day")
    hist_apparent_temperature_max_t1: Optional[float] = Field(None, description="Max apparent temperature (°C) - previous day")
    hist_weather_code_t1: Optional[int] = Field(None, description="Weather code - previous day")
    
    # Weather features (historical aggregations)
    hist_precip_sum_7d: Optional[float] = Field(None, description="7-day precipitation sum (mm)")
    hist_precip_sum_3d: Optional[float] = Field(None, description="3-day precipitation sum (mm)")
    hist_wind_max_7d: Optional[float] = Field(None, description="7-day max wind speed (km/h)")
    
    # Weather features (forecast - for target day)
    fcst_precipitation_sum: float = Field(..., description="Forecast precipitation sum (mm)")
    fcst_precipitation_hours: Optional[float] = Field(None, description="Forecast precipitation hours")
    fcst_wind_speed_max: float = Field(..., description="Forecast max wind speed (km/h)")
    fcst_wind_gusts_max: Optional[float] = Field(None, description="Forecast max wind gusts (km/h)")
    fcst_pressure_msl_min: Optional[float] = Field(None, description="Forecast min MSL pressure (hPa)")
    fcst_temperature_max: Optional[float] = Field(None, description="Forecast max temperature (°C)")
    fcst_relative_humidity_mean: Optional[float] = Field(None, description="Forecast mean relative humidity (%)")
    fcst_cloud_cover_max: Optional[float] = Field(None, description="Forecast max cloud cover (%)")
    fcst_dew_point_mean: Optional[float] = Field(None, description="Forecast mean dew point (°C)")
    fcst_cape_max: Optional[float] = Field(None, description="Forecast max CAPE (J/kg)")
    
    # Flood risk
    mean_flood_risk_score: Optional[float] = Field(None, description="Mean flood risk score for LGU")
    
    @validator('date')
    def validate_date(cls, v):
        try:
            pd.to_datetime(v)
            return v
        except:
            raise ValueError(f"Invalid date format: {v}. Expected YYYY-MM-DD")
    
    @validator('lgu_name')
    def validate_lgu(cls, v):
        if v not in REVERSE_LOCATION_MAPPING:
            raise ValueError(f"Invalid LGU: {v}. Must be one of {list(REVERSE_LOCATION_MAPPING.keys())}")
        return v

class PredictionOutput(BaseModel):
    """Output schema for prediction response"""
    date: str
    lgu_name: str
    suspension_predicted: int = Field(..., description="0 or 1")
    suspension_probability: float = Field(..., description="Probability of suspension [0.0-1.0]")
    confidence_level: str = Field(..., description="Low, Medium, or High")
    model_version: str
    prediction_timestamp: str

print("✅ Pydantic models defined")
print(f"   Input fields: {len(PredictionInput.__fields__)}")
print(f"   Output fields: {len(PredictionOutput.__fields__)}")

In [ ]:
# Cell 1.2: Create Preprocessing Pipeline
class PredictionPipeline:
    """
    Production-ready prediction pipeline
    """
    
    def __init__(self, model, metadata, location_mapping):
        self.model = model
        self.metadata = metadata
        self.location_mapping = location_mapping
        self.reverse_location_mapping = {v: k for k, v in location_mapping.items()}
        self.expected_features = metadata['selected_features']
        self.model_version = f"{metadata['best_model']}_v1.0"
        
    def preprocess_input(self, input_data: PredictionInput) -> pd.DataFrame:
        """
        Convert PredictionInput to model-ready DataFrame
        """
        # Parse date
        date_obj = pd.to_datetime(input_data.date)
        
        # Extract calendar features from date
        year = date_obj.year
        month = date_obj.month
        day = date_obj.day
        day_of_week = date_obj.dayofweek
        
        # Rainy season (June-November)
        is_rainy_season = 1 if month in [6, 7, 8, 9, 10, 11] else 0
        
        # School year start (June = 0, May = 11)
        if month >= 6:
            month_from_sy_start = month - 6
        else:
            month_from_sy_start = month + 6
        
        # Get LGU ID
        lgu_id = self.reverse_location_mapping[input_data.lgu_name]
        
        # Get flood risk (use default if not provided)
        mean_flood_risk_score = input_data.mean_flood_risk_score or 2.5
        
        # Create feature dictionary
        features = {
            'year': year,
            'month': month,
            'day': day,
            'day_of_week': day_of_week,
            'is_rainy_season': is_rainy_season,
            'month_from_sy_start': month_from_sy_start,
            'is_holiday': input_data.is_holiday or 0,
            'is_school_day': input_data.is_school_day or 1,
            'lgu_id': lgu_id,
            'mean_flood_risk_score': mean_flood_risk_score,
            
            # Historical weather (t-1)
            'hist_precipitation_sum_t1': input_data.hist_precipitation_sum_t1,
            'hist_wind_speed_max_t1': input_data.hist_wind_speed_max_t1,
            'hist_wind_gusts_max_t1': input_data.hist_wind_gusts_max_t1 or 0,
            'hist_pressure_msl_min_t1': input_data.hist_pressure_msl_min_t1 or 1013,
            'hist_temperature_max_t1': input_data.hist_temperature_max_t1 or 30,
            'hist_relative_humidity_mean_t1': input_data.hist_relative_humidity_mean_t1 or 75,
            'hist_cloud_cover_max_t1': input_data.hist_cloud_cover_max_t1 or 50,
            'hist_dew_point_mean_t1': input_data.hist_dew_point_mean_t1 or 25,
            'hist_apparent_temperature_max_t1': input_data.hist_apparent_temperature_max_t1 or 32,
            'hist_weather_code_t1': input_data.hist_weather_code_t1 or 0,
            
            # Historical aggregations
            'hist_precip_sum_7d': input_data.hist_precip_sum_7d or input_data.hist_precipitation_sum_t1 * 3,
            'hist_precip_sum_3d': input_data.hist_precip_sum_3d or input_data.hist_precipitation_sum_t1 * 1.5,
            'hist_wind_max_7d': input_data.hist_wind_max_7d or input_data.hist_wind_speed_max_t1 * 1.2,
            
            # Forecast weather
            'fcst_precipitation_sum': input_data.fcst_precipitation_sum,
            'fcst_precipitation_hours': input_data.fcst_precipitation_hours or 0,
            'fcst_wind_speed_max': input_data.fcst_wind_speed_max,
            'fcst_wind_gusts_max': input_data.fcst_wind_gusts_max or 0,
            'fcst_pressure_msl_min': input_data.fcst_pressure_msl_min or 1013,
            'fcst_temperature_max': input_data.fcst_temperature_max or 30,
            'fcst_relative_humidity_mean': input_data.fcst_relative_humidity_mean or 75,
            'fcst_cloud_cover_max': input_data.fcst_cloud_cover_max or 50,
            'fcst_dew_point_mean': input_data.fcst_dew_point_mean or 25,
            'fcst_cape_max': input_data.fcst_cape_max or 0,
        }
        
        # Create DataFrame with expected feature order
        df = pd.DataFrame([features])
        df = df[self.expected_features]
        
        return df
    
    def predict(self, input_data: PredictionInput) -> PredictionOutput:
        """
        Make prediction and return structured output
        """
        # Preprocess
        X = self.preprocess_input(input_data)
        
        # Predict
        y_pred = self.model.predict(X)[0]
        y_proba = self.model.predict_proba(X)[0, 1]
        
        # Determine confidence level
        if y_proba < 0.3 or y_proba > 0.7:
            confidence = "High"
        elif y_proba < 0.4 or y_proba > 0.6:
            confidence = "Medium"
        else:
            confidence = "Low"
        
        # Create output
        output = PredictionOutput(
            date=input_data.date,
            lgu_name=input_data.lgu_name,
            suspension_predicted=int(y_pred),
            suspension_probability=float(y_proba),
            confidence_level=confidence,
            model_version=self.model_version,
            prediction_timestamp=datetime.now().isoformat()
        )
        
        return output

# Initialize pipeline
pipeline = PredictionPipeline(best_model, metadata, LOCATION_MAPPING)
print("✅ Prediction pipeline created")
print(f"   Model version: {pipeline.model_version}")
print(f"   Expected features: {len(pipeline.expected_features)}")

In [ ]:
# Cell 1.3: Test the Pipeline
print("="*80)
print("TESTING PREDICTION PIPELINE")
print("="*80)

# Create test input (heavy rain scenario)
test_input = PredictionInput(
    date="2025-07-15",
    lgu_name="Manila",
    is_holiday=0,
    is_school_day=1,
    hist_precipitation_sum_t1=50.0,
    hist_wind_speed_max_t1=45.0,
    hist_wind_gusts_max_t1=60.0,
    fcst_precipitation_sum=80.0,
    fcst_wind_speed_max=55.0,
    fcst_wind_gusts_max=70.0,
    mean_flood_risk_score=4.0
)

print("\nTest Input (Heavy Rain Scenario):")
print(f"  Date: {test_input.date}")
print(f"  LGU: {test_input.lgu_name}")
print(f"  Historical Precipitation: {test_input.hist_precipitation_sum_t1} mm")
print(f"  Forecast Precipitation: {test_input.fcst_precipitation_sum} mm")
print(f"  Flood Risk: {test_input.mean_flood_risk_score}")

# Make prediction
result = pipeline.predict(test_input)

print("\n" + "="*80)
print("PREDICTION RESULT")
print("="*80)
print(f"  Suspension Predicted: {'YES' if result.suspension_predicted else 'NO'}")
print(f"  Probability: {result.suspension_probability:.2%}")
print(f"  Confidence: {result.confidence_level}")
print(f"  Model Version: {result.model_version}")
print(f"  Timestamp: {result.prediction_timestamp}")

print("\n✅ Pipeline test successful!")

## Section 2: Batch Prediction Capabilities

In [ ]:
# Cell 2.1: Batch Prediction Function
def batch_predict(pipeline: PredictionPipeline, input_list: List[PredictionInput]) -> pd.DataFrame:
    """
    Make predictions for multiple inputs
    """
    results = []
    
    for input_data in input_list:
        try:
            output = pipeline.predict(input_data)
            results.append(output.dict())
        except Exception as e:
            print(f"Error processing {input_data.date} - {input_data.lgu_name}: {e}")
            continue
    
    return pd.DataFrame(results)

print("✅ Batch prediction function defined")

## Section 3: Save Production Artifacts

In [ ]:
# Cell 3.1: Save Production Model Package
print("="*80)
print("SAVING PRODUCTION ARTIFACTS")
print("="*80)

# 1. Save model
model_prod_path = PRODUCTION_DIR / 'model.pkl'
joblib.dump(best_model, model_prod_path)
print(f"\n✅ Model saved: {model_prod_path}")

# 2. Save pipeline
pipeline_path = PRODUCTION_DIR / 'pipeline.pkl'
joblib.dump(pipeline, pipeline_path)
print(f"✅ Pipeline saved: {pipeline_path}")

# 3. Save metadata with production info
production_metadata = {
    **metadata,
    'model_version': pipeline.model_version,
    'production_date': datetime.now().isoformat(),
    'location_mapping': LOCATION_MAPPING,
    'model_file': 'model.pkl',
    'pipeline_file': 'pipeline.pkl',
    'input_schema': list(PredictionInput.__fields__.keys()),
    'output_schema': list(PredictionOutput.__fields__.keys())
}

metadata_prod_path = PRODUCTION_DIR / 'metadata.json'
with open(metadata_prod_path, 'w') as f:
    json.dump(production_metadata, f, indent=2)
print(f"✅ Metadata saved: {metadata_prod_path}")

# 4. Save example input/output
example_input = test_input.dict()
example_output = result.dict()

examples = {
    'input_example': example_input,
    'output_example': example_output
}

examples_path = PRODUCTION_DIR / 'examples.json'
with open(examples_path, 'w') as f:
    json.dump(examples, f, indent=2)
print(f"✅ Examples saved: {examples_path}")

print("\n" + "="*80)
print("PRODUCTION ARTIFACTS READY")
print("="*80)
print(f"\nLocation: {PRODUCTION_DIR}")
print(f"\nFiles created:")
for file in PRODUCTION_DIR.glob('*'):
    size_mb = file.stat().st_size / (1024 * 1024)
    print(f"  - {file.name}: {size_mb:.2f} MB")

## Section 4: Generate FastAPI Code

In [ ]:
# Cell 4.1: Generate FastAPI Application Code
fastapi_code = '''# Production FastAPI Application - School Suspension Prediction
# Generated from 04_production_deployment.ipynb

from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field, validator
from typing import Optional, List
from datetime import datetime
import joblib
import json
from pathlib import Path

# Initialize FastAPI
app = FastAPI(
    title="School Suspension Prediction API",
    description="Predict school suspensions based on weather and calendar data",
    version="1.0.0"
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load model and metadata on startup
MODELS_DIR = Path("model-training/models/production")

with open(MODELS_DIR / "metadata.json", "r") as f:
    metadata = json.load(f)

pipeline = joblib.load(MODELS_DIR / "pipeline.pkl")

print(f"✅ Model loaded: {metadata['model_version']}")

# Import Pydantic models from pipeline
# (In production, copy the PredictionInput and PredictionOutput classes here)

@app.get("/")
async def root():
    return {
        "message": "School Suspension Prediction API",
        "model_version": metadata["model_version"],
        "model_type": metadata["best_model"],
        "status": "healthy"
    }

@app.get("/health")
async def health_check():
    return {
        "status": "healthy",
        "timestamp": datetime.now().isoformat(),
        "model_version": metadata["model_version"]
    }

@app.get("/model/info")
async def model_info():
    return {
        "model_type": metadata["best_model"],
        "version": metadata["model_version"],
        "f2_score": metadata["best_f2"],
        "recall": metadata["best_recall"],
        "precision": metadata["best_precision"],
        "features_count": metadata["final_feature_count"],
        "production_date": metadata["production_date"]
    }

@app.post("/predict", response_model=PredictionOutput)
async def predict(input_data: PredictionInput):
    """
    Make a suspension prediction for a single date and LGU
    """
    try:
        result = pipeline.predict(input_data)
        return result
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.post("/predict/batch")
async def predict_batch(input_list: List[PredictionInput]):
    """
    Make predictions for multiple inputs
    """
    try:
        results = []
        for input_data in input_list:
            result = pipeline.predict(input_data)
            results.append(result.dict())
        return {"predictions": results, "count": len(results)}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# Save FastAPI code
web_dir = Path('../../web')
fastapi_path = web_dir / 'app_production.py'

with open(fastapi_path, 'w') as f:
    f.write(fastapi_code)

print("✅ FastAPI application code generated")
print(f"   Location: {fastapi_path}")
print(f"\nTo run the API:")
print(f"  cd {web_dir}")
print(f"  python app_production.py")

## Section 5: Documentation & README

In [ ]:
# Cell 5.1: Generate Production README
readme_content = f'''# School Suspension Prediction - Production Model

## Model Information
- **Model Type**: {metadata['best_model']}
- **Version**: {pipeline.model_version}
- **Performance**:
  - F2 Score: {metadata['best_f2']:.4f}
  - Recall: {metadata['best_recall']:.4f}
  - Precision: {metadata['best_precision']:.4f}
- **Features**: {metadata['final_feature_count']}
- **Production Date**: {datetime.now().strftime('%Y-%m-%d')}

## Files
- `model.pkl`: Trained EasyEnsemble model
- `pipeline.pkl`: Complete prediction pipeline with preprocessing
- `metadata.json`: Model metadata and configuration
- `examples.json`: Input/output examples
- `README.md`: This file

## Quick Start

### Load the Pipeline
```python
import joblib
from pathlib import Path

# Load pipeline
pipeline = joblib.load('production/pipeline.pkl')

# Make prediction
from pydantic import BaseModel

input_data = PredictionInput(
    date="2025-07-15",
    lgu_name="Manila",
    hist_precipitation_sum_t1=50.0,
    hist_wind_speed_max_t1=45.0,
    fcst_precipitation_sum=80.0,
    fcst_wind_speed_max=55.0
)

result = pipeline.predict(input_data)
print(f"Suspension Predicted: {{result.suspension_predicted}}")
print(f"Probability: {{result.suspension_probability:.2%}}")
```

### Run FastAPI Server
```bash
cd ../../web
python app_production.py
```

Visit: http://localhost:8000/docs for interactive API documentation

## API Endpoints

### Health Check
```bash
curl http://localhost:8000/health
```

### Model Info
```bash
curl http://localhost:8000/model/info
```

### Single Prediction
```bash
curl -X POST http://localhost:8000/predict \\
  -H "Content-Type: application/json" \\
  -d '{{
    "date": "2025-07-15",
    "lgu_name": "Manila",
    "hist_precipitation_sum_t1": 50.0,
    "hist_wind_speed_max_t1": 45.0,
    "fcst_precipitation_sum": 80.0,
    "fcst_wind_speed_max": 55.0
  }}'
```

## Input Requirements

### Required Fields
- `date`: Date in YYYY-MM-DD format
- `lgu_name`: LGU name (Manila, Quezon City, etc.)
- `hist_precipitation_sum_t1`: Yesterday's precipitation (mm)
- `hist_wind_speed_max_t1`: Yesterday's max wind speed (km/h)
- `fcst_precipitation_sum`: Today's forecast precipitation (mm)
- `fcst_wind_speed_max`: Today's forecast wind speed (km/h)

### Optional Fields
All other weather and calendar features have defaults

## Monitoring

Monitor these metrics in production:
- Prediction latency
- Prediction confidence distribution
- Actual vs predicted suspension rate
- Feature drift (weather patterns changing over time)

## Retraining

Retrain when:
- Model performance degrades (F2 < 0.50)
- New suspension data available (quarterly)
- Weather patterns change significantly
- New LGUs need to be added

## Support
For issues or questions, refer to the training notebooks:
- `02_eda_and_core_model_all_features.ipynb`: Model training
- `03_final_test_evaluation.ipynb`: Model validation
- `04_production_deployment.ipynb`: This deployment process
'''

readme_path = PRODUCTION_DIR / 'README.md'
with open(readme_path, 'w') as f:
    f.write(readme_content)

print("✅ Production README generated")
print(f"   Location: {readme_path}")

## Section 6: Final Summary

In [ ]:
# Cell 6.1: Production Deployment Summary
print("="*80)
print("PRODUCTION DEPLOYMENT COMPLETE")
print("="*80)

print(f"\n📦 Production Package Location: {PRODUCTION_DIR}")

print(f"\n📄 Files Generated:")
for file in sorted(PRODUCTION_DIR.glob('*')):
    size_mb = file.stat().st_size / (1024 * 1024)
    print(f"   ✅ {file.name}: {size_mb:.2f} MB")

print(f"\n🔧 Model Details:")
print(f"   Type: {metadata['best_model']}")
print(f"   Version: {pipeline.model_version}")
print(f"   F2 Score: {metadata['best_f2']:.4f}")
print(f"   Features: {metadata['final_feature_count']}")

print(f"\n🚀 Next Steps:")
print(f"   1. Review the production README: {readme_path}")
print(f"   2. Test the FastAPI endpoint: cd ../../web && python app_production.py")
print(f"   3. Deploy to your hosting platform (Supabase, AWS, Azure, etc.)")
print(f"   4. Set up monitoring and alerting")
print(f"   5. Schedule retraining pipeline")

print(f"\n" + "="*80)
print("🎉 PRODUCTION READY!")
print("="*80)